First, activate conda environment:
```bash
conda activate NGTR
```

In [1]:
import os
# Go outside the src directory
os.chdir("../Image2Table_LLM")
current_dir = os.getcwd()
print("Current directory:", current_dir)

Current directory: /Users/sarah_shoilee/codeProjects/stamboekn_KE/Image2Table_LLM


In [2]:
from LLM import call_LLM
from LLM_key import llm_model
from parse import extract_HTML, format_td
from prompt import tsr_html_prompt, tsr_html_decomposition
from metric import TEDS

In [3]:
image_path = "../data/images/NL-HaNA_2.10.50_45_0355.jpg"
image_name = os.path.basename(image_path)

In [4]:
llm_html = call_LLM(image_path, prompt=tsr_html_prompt, model_name=llm_model, temperature=0)
llm_html = extract_HTML(llm_html)

```html
<table>
  <thead>
    <tr>
      <td>Namen en Toenamen</td>
      <td>Namen der Ouders, datum van Geboorte, Geboorteplaats en laatste Woonsplaats, en of dezelfde gehuwd zijn en kinderen hebben</td>
      <td>Datums van aanstellingen in onderscheidene Graden, van overgang naar andere korpsen, van bekomen Pensioen of Demissie en van Overlijden</td>
      <td>In iedere graad gediend</td>
      <td>Gedane veldtogten, bekomen wonden, uitstekende daden of bijzondere verrigtingen</td>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td> Wolff </td>
      <td> Vader Jacob Young Moeder Catharina Cantolt Geboren den 8e October 1773 Geboorteplaats Schoonhoven Laatste Woonsplaats Buille </td>
      <td></td>
      <td></td>
      <td></td>
    </tr>
    <tr>
      <td> van Schack </td>
      <td> Vader Cornst Hendrik Moeder Maria Cantolt Geboren den 22e December 1771 Geboorteplaats Sloge Laatste Woonsplaats Sloge </td>
      <td></td>
      <td></td>
      <td></td>
    </tr>
  </tbody>
</ta

In [5]:
# save the html
with open(os.path.join("../data/tables/html", image_name.replace('.jpg', '.html')), 'w', encoding='utf-8') as f:
    f.write(llm_html)

In [6]:
def calculate_TEDS(ground_truth_html, predicted_html):
    predicted_html = format_td(predicted_html)
    ground_truth_html = format_td(ground_truth_html)

    teds = TEDS(structure_only=False)
    teds_score = teds.evaluate(ground_truth_html, predicted_html)

    teds_struct = TEDS(structure_only=True)
    teds_struct_score = teds_struct.evaluate(ground_truth_html, predicted_html)
    
    print(f"TEDS: {teds_score:.4f}")
    print(f"TEDS-Struct: {teds_struct_score:.4f}")

    return teds_score, teds_struct_score

In [7]:
with open(os.path.join("../data/labels", image_name.replace('.jpg', '.html')), 'r', encoding='utf-8') as f:
    label_html = f.read()

calculate_TEDS(label_html, llm_html)


TEDS: 0.8801
TEDS-Struct: 0.9286


(0.880062006735847, 0.9285714285714286)

### Information Extraction

In [8]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(llm_html, "html.parser")
table = soup.find("table")

In [9]:
table

<table><thead><tr><td>Namen en Toenamen</td><td>Namen der Ouders, datum van Geboorte, Geboorteplaats en laatste Woonsplaats, en of dezelfde gehuwd zijn en kinderen hebben</td><td>Datums van aanstellingen in onderscheidene Graden, van overgang naar andere korpsen, van bekomen Pensioen of Demissie en van Overlijden</td><td>In iedere graad gediend</td><td>Gedane veldtogten, bekomen wonden, uitstekende daden of bijzondere verrigtingen</td></tr></thead><tbody><tr><td> Wolff </td><td> Vader Jacob Young Moeder Catharina Cantolt Geboren den 8e October 1773 Geboorteplaats Schoonhoven Laatste Woonsplaats Buille </td><td></td><td></td><td></td></tr><tr><td> van Schack </td><td> Vader Cornst Hendrik Moeder Maria Cantolt Geboren den 22e December 1771 Geboorteplaats Sloge Laatste Woonsplaats Sloge </td><td></td><td></td><td></td></tr></tbody></table>

In [10]:
import re
import json

persons = []

for row in table.find_all("tr"):
    cells = row.find_all("td")

    if not cells:
        continue  # skip header or empty rows

    person = {}
    for cell in cells:
        # preserve breaks as \n
        text = cell.get_text(separator="\n", strip=True)
        
        vader_match = re.search(r'Vader\s+([^\n,<]+)', text, re.IGNORECASE)
        moeder_match = re.search(r'Moeder\s+([^\n,<]+)', text, re.IGNORECASE)
        geboorte_datum_match = re.search(r'Geboren\s*Den\s*([^\n,<]+)', text, re.IGNORECASE)
        geboorte_plaats_match = re.search(r'Geboortplaats\s*([^\n,<]+)', text, re.IGNORECASE)
        laatste_woonplaats_match = re.search(r'Laatste\s*Woonplaats\s*([^\n,<]+)', text, re.IGNORECASE)
        
        if vader_match:
            person['vader'] = {'value': vader_match.group(1).strip(), 'cell': cell.get('id')}
        if moeder_match:
            person['moeder'] = {'value': moeder_match.group(1).strip(), 'cell': cell.get('id')}
        if geboorte_datum_match:
            person['geboorte_datum'] = {'value': geboorte_datum_match.group(1).strip(), 'cell': cell.get('id')}
        if geboorte_plaats_match:
            person['geboorte_plaats'] = {'value': geboorte_plaats_match.group(1).strip(), 'cell': cell.get('id')}
        if laatste_woonplaats_match:
            person['laatste_woonplaats'] = {'value': laatste_woonplaats_match.group(1).strip(), 'cell': cell.get('id')}

    if person:
        persons.append(person)

json_obj = {"persons": persons}
print(json.dumps(json_obj, indent=2, ensure_ascii=False))

{
  "persons": [
    {
      "vader": {
        "value": "Jacob Young Moeder Catharina Cantolt Geboren den 8e October 1773 Geboorteplaats Schoonhoven Laatste Woonsplaats Buille",
        "cell": null
      },
      "moeder": {
        "value": "Catharina Cantolt Geboren den 8e October 1773 Geboorteplaats Schoonhoven Laatste Woonsplaats Buille",
        "cell": null
      },
      "geboorte_datum": {
        "value": "8e October 1773 Geboorteplaats Schoonhoven Laatste Woonsplaats Buille",
        "cell": null
      }
    },
    {
      "vader": {
        "value": "Cornst Hendrik Moeder Maria Cantolt Geboren den 22e December 1771 Geboorteplaats Sloge Laatste Woonsplaats Sloge",
        "cell": null
      },
      "moeder": {
        "value": "Maria Cantolt Geboren den 22e December 1771 Geboorteplaats Sloge Laatste Woonsplaats Sloge",
        "cell": null
      },
      "geboorte_datum": {
        "value": "22e December 1771 Geboorteplaats Sloge Laatste Woonsplaats Sloge",
        "cell":

In [11]:
with open(f"../data/json/{image_name}.json", "w", encoding='utf-8') as json_file:
    json.dump(json_obj, json_file, ensure_ascii=False, indent=2)#!/usr/bin/env python3


### KG construction



In [12]:
# load json from file
with open(f"../data/json/{image_name}.json", "r", encoding="utf-8") as f:
    json_obj = json.load(f)

In [13]:
# CONSTRUCT ASSERSION TRIPLES
from rdflib import Graph, ConjunctiveGraph, Namespace, URIRef, Literal, RDF

FOAF = Namespace("http://xmlns.com/foaf/0.1/")
EX = Namespace("http://example.org/ontology/")
PROV = Namespace("http://www.w3.org/ns/prov#")

cg = ConjunctiveGraph()
cg.bind("foaf", FOAF)
cg.bind("ex", EX)
cg.bind("prov", PROV)

# Mapping from json keys to RDF predicates
predicate_map = {
    "vader": EX.vader,
    "moeder": EX.moeder,
    "geboorte_datum": EX.geboorteDatum,
    "geboorte_plaats": EX.geboortePlaats,
    "laatste_woonplaats": EX.laatsteWoonplaats
}

for idx, person in enumerate(json_obj["persons"], start=1):
    person_uri = URIRef(f"http://example.org/person/{idx}")
    assertion_graph_uri = URIRef("http://example.org/assertion")
    assertion_graph = Graph(store=cg.store, identifier=assertion_graph_uri)
    assertion_graph.add((person_uri, RDF.type, FOAF.Person))

    provenance_graph_uri = URIRef("http://example.org/provenance")
    provenance_graph = Graph(store=cg.store, identifier=provenance_graph_uri)

    for key, value_dict in person.items():
        value = value_dict["value"]
        cell_id = value_dict["cell"]
        
        predicate = predicate_map.get(key)
        
        # if cell_id is null, no named graph can be created
        if not cell_id:
            assertion_graph.add((person_uri, predicate, Literal(value)))
            continue
        
        if predicate:
            # Named graph for each cell
            graph_uri = URIRef(f"http://example.org/graph/{cell_id}")
            ng = Graph(store=cg.store, identifier=graph_uri)
            ng.add((person_uri, predicate, Literal(value)))
            

cg.serialize(f"../data/triples/{image_name}_assersion.trig", format='trig')


/var/folders/jg/8kz6nb554_3dz1_0swrmdw480000gn/T/ipykernel_16304/1704695848.py:8: DeprecationWarning: ConjunctiveGraph is deprecated, use Dataset instead.
  cg = ConjunctiveGraph()


<Graph identifier=N90ceb5ce27b7484fae4ededc5a57bc2a (<class 'rdflib.graph.ConjunctiveGraph'>)>